# Curso de aprendizaje automatizado
PCIC, UNAM

Machine Learning

Rodrigo S. Cortez Madrigal

<img src="https://pcic.posgrado.unam.mx/wp-content/uploads/Ciencia-e-Ingenieria-de-la-Computacion_color.png" alt="Logo PCIC" width="128" />   

### Tarea 2: Regresión y clasificación lineal

A partir del conjunto de datos Automobile Dataset realiza la regresión de los precios de automóviles con las siguientes variantes:

- a. Mínimos cuadrados con expansión polinomial de diferentes grados.
- b. Mínimos cuadrados con expansión polinomial de grado 20 y penalización por norma l1 y l2 con diferentes valores de λ.
- c. Mínimos cuadrados con expansión polinomial de grado 2 y selección de atributos.

Grafica el error cuadrático medio en entrenamiento y validación con respecto al grado del polinomio,
valor de λ y número de atributos. Todos los modelos deberán ser evaluados con 10 repeticiones de
validación cruzada de 5 particiones. Selecciona uno de los modelos y reporta su desempeño en el
conjunto de prueba.

In [1]:
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import RepeatedKFold, cross_val_score

import plotly.express as px
import plotly.graph_objects as go

from tqdm import tqdm
import numpy as np
from joblib import Parallel, delayed

In [2]:
# Disable warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")

## Automovil Dataset

El conjunto de datos Automobile Dataset contiene información sobre diferentes automóviles, incluyendo el precio. El objetivo es predecir el precio de los automóviles a partir de las características de los mismos.

In [3]:
# Load the data

data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data', header=None, sep=',', engine='python', na_values='?')

data.columns = ['symboling', 'normalized_losses', 'maker', 'fuel_type', 'aspiration', 'num_doors', 'body_style', 'drive_wheels', 'engine_location', 'wheel_base', 'length', 'width', 'height', 'curb_weight', 'engine_type', 'num_cylinders', 'engine_size', 'fuel_system', 'bore', 'stroke', 'compression_ratio', 'horsepower', 'peak_rpm', 'city_mpg', 'highway_mpg', 'price']

In [4]:
data.describe()

,symboling,normalized_losses,wheel_base,length,width,height,curb_weight,engine_size,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price
count,205.000000,164.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,201.000000,201.000000,205.000000,203.000000,203.000000,205.000000,205.000000,201.000000
mean,0.834146,122.000000,98.756585,174.049268,65.907805,53.724878,2555.565854,126.907317,3.329751,3.255423,10.142537,104.256158,5125.369458,25.219512,30.751220,13207.129353
std,1.245307,35.442168,6.021776,12.337289,2.145204,2.443522,520.680204,41.642693,0.273539,0.316717,3.972040,39.714369,479.334560,6.542142,6.886443,7947.066342
min,-2.000000,65.000000,86.600000,141.100000,60.300000,47.800000,1488.000000,61.000000,2.540000,2.070000,7.000000,48.000000,4150.000000,13.000000,16.000000,5118.000000
25%,0.000000,94.000000,94.500000,166.300000,64.100000,52.000000,2145.000000,97.000000,3.150000,3.110000,8.600000,70.000000,4800.000000,19.000000,25.000000,7775.000000
50%,1.000000,115.000000,97.000000,173.200000,65.500000,54.100000,2414.000000,120.000000,3.310000,3.290000,9.000000,95.000000,5200.000000,24.000000,30.000000,10295.000000
75%,2.000000,150.000000,102.400000,183.100000,66.900000,55.500000,2935.000000,141.000000,3.590000,3.410000,9.400000,116.000000,5500.000000,30.000000,34.000000,16500.000000
max,3.000000,256.000000,120.900000,208.100000,72.300000,59.800000,4066.000000,326.000000,3.940000,4.170000,23.000000,288.000000,6600.000000,49.000000,54.000000,45400.000000


In [5]:
# Missing values by column

pd.DataFrame(data.isnull().sum())

,0
symboling,0
normalized_losses,41
maker,0
fuel_type,0
aspiration,0
num_doors,2
body_style,0
drive_wheels,0
engine_location,0
wheel_base,0


In [6]:
total_faltantes = len(data) - len(data.dropna())
print("Total de datos faltantes:", total_faltantes)
porcentaje_faltantes = total_faltantes/len(data)*100
print("Porcentaje de datos faltantes:", porcentaje_faltantes, "%")

x = [porcentaje_faltantes, 100-porcentaje_faltantes]
fig = px.pie(values=x, names=["Datos faltantes", "Datos no faltantes"], title="Porcentaje de datos faltantes")
fig.show()

Total de datos faltantes: 46
Porcentaje de datos faltantes: 22.439024390243905 %


In [7]:
# Estrategias para manejar los datos faltantes

# 1. Eliminar las filas con datos faltantes
dataWN = data.dropna()

In [8]:
# Correct Types

dataWN['price'] = pd.to_numeric(dataWN['price'])

categorical_columns = ['maker', 'fuel_type', 'aspiration', 'num_doors', 'body_style', 
                       'drive_wheels', 'engine_location', 'engine_type', 'num_cylinders', 'fuel_system']

for col in categorical_columns:
    dataWN[col] = dataWN[col].astype('category')

/var/folders/66/nn87wlh11r1_6tdbhw20rbk80000gn/T/ipykernel_71411/2846757769.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/66/nn87wlh11r1_6tdbhw20rbk80000gn/T/ipykernel_71411/2846757769.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/66/nn87wlh11r1_6tdbhw20rbk80000gn/T/ipykernel_71411/2846757769.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [9]:
# Obtener X, y 

X = dataWN[['symboling', 'normalized_losses', 'maker', 'fuel_type', 'aspiration', 'num_doors', 'body_style', 
            'drive_wheels', 'engine_location', 'wheel_base', 'length', 'width', 'height', 'curb_weight', 
            'engine_type', 'num_cylinders', 'engine_size', 'fuel_system', 'bore', 'stroke', 
            'compression_ratio', 'horsepower', 'peak_rpm', 'city_mpg', 'highway_mpg']]
y = dataWN['price']

# Codificar las columnas categóricas en X
for col in X.select_dtypes(include=['category']).columns:
    X[col] = X[col].cat.codes

/var/folders/66/nn87wlh11r1_6tdbhw20rbk80000gn/T/ipykernel_71411/2584131557.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## Selección de atributos

In [10]:
# Calcular la correlación entre las características y el precio
correlation_matrix = X.corrwith(y).sort_values(ascending=False)
correlation_matrix = pd.DataFrame(correlation_matrix).reset_index()
correlation_matrix.columns = ['feature', 'correlation']

# Filtrar características con correlación significativa
correlation_matrix = correlation_matrix[correlation_matrix['correlation'].abs() > 0.1]
correlation_matrix = correlation_matrix.sort_values(by='correlation', ascending=False)

fig = px.bar(correlation_matrix, x='feature', y='correlation', title='Correlation with price')
fig.update_layout(xaxis_title='Feature', yaxis_title='Correlation')
fig.show()

# Seleccionar las 10 características más correlacionadas
selected_features = correlation_matrix.head(5)['feature'].tolist()
print("Top 10 features:", selected_features)

/opt/homebrew/Caskroom/miniforge/base/envs/datascience/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:3045: RuntimeWarning:

invalid value encountered in divide

/opt/homebrew/Caskroom/miniforge/base/envs/datascience/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:3046: RuntimeWarning:

invalid value encountered in divide



Top 10 features: ['curb_weight', 'width', 'engine_size', 'length', 'horsepower']


In [11]:
# Plot prices by make

fig = px.box(dataWN, x='curb_weight', y='price', title='Price by curb_weight')
fig.show()
fig = px.box(dataWN, x='width', y='price', title='Price by width')
fig.show()
fig = px.box(dataWN, x='horsepower', y='price', title='Price by horsepower')
fig.show()

### Regresión Polinomial

$Y = \beta_0 + \beta_1 X + \beta_2 X^2 + \beta_3 X^3 + \epsilon$

La expansión polinomial consiste en transformar la variable independiente X en un conjunto de términos polinómicos, permitiendo que la regresión modele relaciones no lineales.
- Si usamos un polinomio de grado 1, obtenemos una regresión lineal simple.
- Si usamos un polinomio de grado 2 (cuadrático), el modelo puede capturar curvaturas en los datos.
- A medida que aumentamos el grado, el modelo se vuelve más flexible, pero también más propenso a sobreajustarse.

Grafica el error cuadrático medio en entrenamiento y validación con respecto al grado del polinomio,
valor de λ y número de atributos. Todos los modelos deberán ser evaluados con 10 repeticiones de
validación cruzada de 5 particiones. Selecciona uno de los modelos y reporta su desempeño en el
conjunto de prueba.

In [ ]:

def PolynomialTrainAndEval(degree, X_train, X_test, y_train, y_test, model):
    """
    Entrena y evalúa un modelo de regresión polinómica.

    Args:
        degree (int): Grado del polinomio.
        X_train (DataFrame): Datos de entrenamiento.
        X_test (DataFrame): Datos de prueba.
        y_train (Series): Etiquetas de entrenamiento.
        y_test (Series): Etiquetas de prueba.
        model: Modelo base.
    Returns:
        dict: Diccionario con los resultados de la evaluación.

    Scores:
        mse: Error cuadrático medio.
        r2: Coeficiente de determinación R^2.
        cross_val_mean: Media de la validación cruzada.
        cross_val_std: Desviación estándar de la validación cruzada.

    Descripción de la validación cruzada:
        rkf: RepeatedKFold(n_splits=5, n_repeats=10, random_state=42)
        n_splits=5: Divide los datos en 5 particiones (folds) en cada repetición.
        n_repeats=10: Repite el proceso de partición 10 veces, generando diferentes divisiones en cada repetición.
        random_state=42: Fija una semilla para garantizar que las divisiones sean reproducibles.
        cross_val_train: Evalúa el modelo utilizando validación cruzada. Obtiene Un array que contiene las puntuaciones obtenidas en cada iteración.
        model: El modelo que se evaluará (en este caso, un pipeline con PolynomialFeatures y el modelo base).
        X_ y y_: Los datos que se usarán para entrenar y validar el modelo.
        cv=rkf: Usa el objeto rkf para definir cómo se dividirán los datos en cada iteración de validación cruzada.
    """
    model = make_pipeline(PolynomialFeatures(degree), model)
    model.fit(X_train, y_train)

    rkf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=42) # Crea un objeto que realiza validación cruzada repetida.

    # Para Train
    y_train_pred = model.predict(X_train)
    mse_train = mean_squared_error(y_train, y_train_pred)
    r2_train = r2_score(y_train, y_train_pred) 
    cross_val_train = cross_val_score(model, X_train, y_train, cv=rkf) # Evalúa el modelo utilizando validación cruzada.
    cross_val_mean_train = cross_val_train.mean()
    cross_val_std_train = cross_val_train.std()

    # Para Test
    y_test_pred = model.predict(X_test)
    mse_test = mean_squared_error(y_test, y_test_pred)
    r2_test = r2_score(y_test, y_test_pred)
    cross_val_test = cross_val_score(model, X_test, y_test, cv=rkf) # Evalúa el modelo utilizando validación cruzada.
    cross_val_mean_test = cross_val_test.mean()
    cross_val_std_test = cross_val_test.std()

    return {
        'degree': degree,
        'mse_train': mse_train,
        'r2_train': r2_train,
        'cross_val_mean_train': cross_val_mean_train,
        'cross_val_std_train': cross_val_std_train,
        'mse_test': mse_test,
        'r2_test': r2_test,
        'cross_val_mean_test': cross_val_mean_test,
        'cross_val_std_test': cross_val_std_test
    }

#### mse: Error cuadrático medio.

El error cuadrático medio (MSE) es una medida de la calidad de un estimador. Se define como la media de los cuadrados de las diferencias entre los valores observados y los valores predichos por el modelo.

#### r2: Coeficiente de determinación R^2.

El coeficiente de determinación R^2 es una medida estadística que representa la proporción de la varianza de la variable dependiente que es predecible a partir de la variable independiente. Un valor de R^2 cercano a 1 indica que el modelo explica bien la variabilidad de los datos.

#### Validación cruzada

La validación cruzada es una técnica de evaluación de modelos que consiste en dividir el conjunto de datos en varias particiones (o "folds"). El modelo se entrena en una parte de los datos y se evalúa en la parte restante. Este proceso se repite varias veces, y los resultados se promedian para obtener una estimación más robusta del rendimiento del modelo.

El promedio de la validación cruzada se calcula como:
$$
\text{cross\_val\_mean} = \frac{1}{k} \sum_{i=1}^{k} \text{mse}_i
$$
donde $k$ es el número de particiones y $\text{mse}_i$ es el error cuadrático medio en la i-ésima partición.

La desviación estándar de la validación cruzada se calcula como:
$$
\text{cross\_val\_std} = \sqrt{\frac{1}{k-1} \sum_{i=1}^{k} (\text{mse}_i - \text{cross\_val\_mean})^2}
$$
donde $\text{cross\_val\_mean}$ es el promedio de la validación cruzada y $\text{mse}_i$ es el error cuadrático medio en la i-ésima partición.

### A. Mínimos cuadrados con expansión polinomial de diferentes grados.

In [ ]:
# Lista de grados de polinomio
degrees = [1, 2, 3, 4, 5]

X_train, X_test, y_train, y_test = train_test_split(X[selected_features], y, test_size=0.2, random_state=42)

results = Parallel(n_jobs=-1)(
    delayed(PolynomialTrainAndEval)(degree, X_train, X_test, y_train, y_test, LinearRegression())
    for degree in degrees
)

results_df = pd.DataFrame(results)

In [43]:
results_df

,degree,mse_train,r2_train,cross_val_mean_train,cross_val_std_train,mse_test,r2_test,cross_val_mean_test,cross_val_std_test
0,1,5.789976e+06,0.848955,0.797972,0.093727,6.466301e+06,0.636615,0.538396,0.813883
1,2,3.226951e+06,0.915817,0.712689,0.226291,4.710297e+06,0.735297,-19.467551,49.541306
2,3,8.131799e+05,0.978786,-18.711205,26.880194,3.421764e+07,-0.922919,-45.876961,87.562067
3,4,1.306889e+05,0.996591,-49489.420688,119669.912248,1.009126e+09,-55.709600,-65.718306,140.385630
4,5,1.145006e+05,0.997013,-82179.649822,206067.593725,2.426913e+09,-135.384554,-99.119472,252.279476


In [41]:
# Graficar los resultados
"""
'degree': degree,
'mse_train': mse_train,
'r2_train': r2_train,
'cross_val_mean_train': cross_val_mean_train,
'cross_val_std_train': cross_val_std_train,
'mse_test': mse_test,
'r2_test': r2_test,
'cross_val_mean_test': cross_val_mean_test,
'cross_val_std_test': cross_val_std_test
"""

fig = go.Figure()
fig.add_trace(go.Scatter(x=results_df['degree'], y=results_df['mse_train'], mode='lines+markers', name='MSE Train'))
fig.add_trace(go.Scatter(x=results_df['degree'], y=results_df['r2_train'], mode='lines+markers', name='R2 Train'))
fig.add_trace(go.Scatter(x=results_df['degree'], y=results_df['cross_val_mean_train'], mode='lines+markers', name='Cross Val Mean Train'))
fig.add_trace(go.Scatter(x=results_df['degree'], y=results_df['cross_val_std_train'], mode='lines+markers', name='Cross Val Std Train'))
fig.update_layout(title='Train Results', xaxis_title='Degree', yaxis_title='Score')

fig.add_trace(go.Scatter(x=results_df['degree'], y=results_df['mse_test'], mode='lines+markers', name='MSE Test'))
fig.add_trace(go.Scatter(x=results_df['degree'], y=results_df['r2_test'], mode='lines+markers', name='R2 Test'))
fig.add_trace(go.Scatter(x=results_df['degree'], y=results_df['cross_val_mean_test'], mode='lines+markers', name='Cross Val Mean Test'))
fig.add_trace(go.Scatter(x=results_df['degree'], y=results_df['cross_val_std_test'], mode='lines+markers', name='Cross Val Std Test'))
fig.update_layout(title='Test Results', xaxis_title='Degree', yaxis_title='Score')
fig.show()

### B. Mínimos cuadrados con expansión polinomial de grado 20 y penalización por norma l1 y l2 con diferentes valores de λ.

- La penalización L1 (Lasso) agrega una penalización proporcional a la suma de los valores absolutos de los coeficientes.
- La penalización L2 (Ridge) agrega una penalización proporcional al cuadrado de los coeficientes.

In [ ]:
from sklearn.linear_model import Ridge, Lasso

X_train, X_test, y_train, y_test = train_test_split(X[selected_features], y, test_size=0.2, random_state=42)

# Valores de lambda (alpha en scikit-learn)
lambdas = [0.01, 0.1, 1, 10, 100]

ridge_results = []
lasso_results = []
degree = 20

ridge_results = Parallel(n_jobs=-1)(
    delayed(PolynomialTrainAndEval)(degree, X_train, X_test, y_train, y_test, Ridge(alpha=alpha))
    for alpha in lambdas
)

lasso_results = Parallel(n_jobs=-1)(
    delayed(PolynomialTrainAndEval)(degree, X_train, X_test, y_train, y_test, Lasso(alpha=alpha, max_iter=10000))
    for alpha in lambdas
)

ridge_results = pd.DataFrame(ridge_results)
lasso_results = pd.DataFrame(lasso_results)

# Save the results to CSV
ridge_results.to_csv('ridge_results_20Degree.csv', index=False)
lasso_results.to_csv('lasso_results_20Degree.csv', index=False)

/opt/homebrew/Caskroom/miniforge/base/envs/datascience/lib/python3.13/site-packages/sklearn/linear_model/_ridge.py:254: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/datascience/lib/python3.13/site-packages/sklearn/linear_model/_ridge.py:254: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/datascience/lib/python3.13/site-packages/sklearn/linear_model/_ridge.py:254: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/datascience/lib/python3.13/site-packages/sklearn/linear_model/_ridge.py:254: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/datascience/lib/python3.13/site-packages/

In [18]:
ridge_results

,degree,mse_train,r2_train,cross_val_mean_train,cross_val_std_train,mse_test,r2_test,cross_val_mean_test,cross_val_std_test
0,20,6.470020e+06,0.831215,-1654.463597,4269.997097,1.125004e+07,0.367784,-10410.81292,34587.360838
1,20,6.470020e+06,0.831215,-1654.463597,4269.997097,1.125004e+07,0.367784,-10410.81292,34587.360838
2,20,6.470020e+06,0.831215,-1654.463597,4269.997097,1.125004e+07,0.367784,-10410.81292,34587.360838
3,20,6.470020e+06,0.831215,-1654.463597,4269.997097,1.125004e+07,0.367784,-10410.81292,34587.360838
4,20,6.470020e+06,0.831215,-1654.463597,4269.997097,1.125004e+07,0.367784,-10410.81292,34587.360838


In [19]:
lasso_results

,degree,mse_train,r2_train,cross_val_mean_train,cross_val_std_train,mse_test,r2_test,cross_val_mean_test,cross_val_std_test
0,20,1.001422e+06,0.973876,-53871.575289,365609.421287,1.065048e+07,0.401478,-661.882104,3327.372369
1,20,1.001530e+06,0.973873,-53811.306311,365191.042002,1.064472e+07,0.401802,-664.834666,3344.672286
2,20,1.002505e+06,0.973847,-53293.308219,361599.489134,1.059671e+07,0.404499,-669.878245,3396.812464
3,20,1.002025e+06,0.973860,-54783.027379,372066.522211,1.067668e+07,0.400005,-659.640611,3356.213608
4,20,1.000782e+06,0.973892,-55202.322218,375056.536230,1.063322e+07,0.402448,-677.025335,3460.615109


Recordemos que la Validación Cruzada es una técnica que permite evaluar el rendimiento de un modelo de aprendizaje automático dividiendo el conjunto de datos en varias partes (o "folds"). 

En cada iteración, se entrena el modelo en una parte del conjunto de datos y se evalúa en la parte restante. Esto ayuda a obtener una estimación más robusta del rendimiento del modelo y a evitar el sobreajuste.

In [40]:
# Graficar los resultados us px

ridge_results['lambda'] = [str(alpha) for alpha in lambdas]
lasso_results['lambda'] = [str(alpha) for alpha in lambdas]

fig = px.bar(ridge_results, x='lambda', y=['mse_train', 'r2_train', 'cross_val_mean_train', 'cross_val_std_train'], title='Ridge Results')
fig.update_layout(title='Ridge Results', xaxis_title='Lambda', yaxis_title='Score')
fig.show()


#### Buscando un modelo

In [ ]:
from sklearn.linear_model import Ridge, Lasso

X_train, X_test, y_train, y_test = train_test_split(X[selected_features], y, test_size=0.2, random_state=42)

# Valores de lambda (alpha en scikit-learn)
lambdas = [0.01, 0.1, 1, 10, 100]

linear_results = []
degree = 2
ridge_results = []
lasso_results = []

linear_results = PolynomialTrainAndEval(degree, X_train, X_test, y_train, y_test, LinearRegression())

ridge_results = Parallel(n_jobs=-1)(
    delayed(PolynomialTrainAndEval)(degree, X_train, X_test, y_train, y_test, Ridge(alpha=alpha))
    for alpha in lambdas
)

lasso_results = Parallel(n_jobs=-1)(
    delayed(PolynomialTrainAndEval)(degree, X_train, X_test, y_train, y_test, Lasso(alpha=alpha, max_iter=10000))
    for alpha in lambdas
)

linear_results = pd.DataFrame(linear_results, index=[0])
ridge_results = pd.DataFrame(ridge_results)
lasso_results = pd.DataFrame(lasso_results)

/opt/homebrew/Caskroom/miniforge/base/envs/datascience/lib/python3.13/site-packages/sklearn/linear_model/_ridge.py:215: LinAlgWarning: Ill-conditioned matrix (rcond=9.46775e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/opt/homebrew/Caskroom/miniforge/base/envs/datascience/lib/python3.13/site-packages/sklearn/linear_model/_ridge.py:215: LinAlgWarning: Ill-conditioned matrix (rcond=9.46775e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/opt/homebrew/Caskroom/miniforge/base/envs/datascience/lib/python3.13/site-packages/sklearn/linear_model/_ridge.py:215: LinAlgWarning: Ill-conditioned matrix (rcond=1.17163e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/opt/homebrew/Caskroom/miniforge/base/envs/datascience/lib/python3.13/site-packages/sklearn/linear_model/_ridge.py:215: LinAlgWarning: Ill-conditioned matrix (rcond=1.08935e-17): resul

In [103]:
# Combinar los resultados
finalresults = pd.concat([linear_results, ridge_results, lasso_results], axis=0)
finalresults['model'] = ['Linear'] + ['Ridge'] * len(ridge_results) + ['Lasso'] * len(lasso_results)
finalresults['lambda'] = [''] + [str(alpha) for alpha in lambdas] + [str(alpha) for alpha in lambdas]
finalresults = finalresults[['model', 'lambda', 'degree', 'mse_train', 'r2_train', 'cross_val_mean_train', 'cross_val_std_train', 
                   'mse_test', 'r2_test', 'cross_val_mean_test', 'cross_val_std_test']]
finalresults = finalresults.reset_index(drop=True)

In [104]:
finalresults

,model,lambda,degree,mse_train,r2_train,cross_val_mean_train,cross_val_std_train,mse_test,r2_test,cross_val_mean_test,cross_val_std_test
0,Linear,,2,3.226951e+06,0.915817,0.712689,0.226291,4.710297e+06,0.735297,-19.467551,49.541306
1,Ridge,0.01,2,3.253357e+06,0.915129,0.717732,0.258945,4.242171e+06,0.761604,-5.594404,28.381944
2,Ridge,0.1,2,3.364004e+06,0.912242,0.719949,0.257473,3.958592e+06,0.777540,-1.678757,8.621463
3,Ridge,1,2,3.430853e+06,0.910498,0.776603,0.157263,3.896923e+06,0.781006,-0.810678,4.918955
4,Ridge,10,2,3.509766e+06,0.908440,0.809608,0.103927,3.900218e+06,0.780821,-0.547266,4.068652
5,Ridge,100,2,3.554605e+06,0.907270,0.815300,0.098018,3.915560e+06,0.779958,-0.045719,2.136103
6,Lasso,0.01,2,3.611290e+06,0.905791,0.827513,0.090777,3.535602e+06,0.801311,0.376364,0.883330
7,Lasso,0.1,2,3.611617e+06,0.905783,0.827507,0.090767,3.534472e+06,0.801374,0.376352,0.883577
8,Lasso,1,2,3.615348e+06,0.905685,0.827412,0.090675,3.523432e+06,0.801995,0.377401,0.885326
9,Lasso,10,2,3.688659e+06,0.903773,0.825524,0.089954,3.454617e+06,0.805862,0.372949,0.917701


In [105]:
# Seleccionar el mejor modelo

best_model = finalresults.loc[finalresults['mse_test'].idxmin()]
best_model

model                            Lasso
lambda                              10
degree                               2
mse_train               3688658.515546
r2_train                      0.903773
cross_val_mean_train          0.825524
cross_val_std_train           0.089954
mse_test                3454616.763895
r2_test                       0.805862
cross_val_mean_test           0.372949
cross_val_std_test            0.917701
Name: 9, dtype: object

In [106]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

X_train, X_test, y_train, y_test = train_test_split(X[selected_features], y, test_size=0.2, random_state=42)

degrees = [1, 2, 3, 4, 5]
lambdas = [0.01, 0.1, 1, 10, 100]

# Definimos el pipeline con un paso genérico para el modelo
pipeline = Pipeline([
    ('polynomialfeatures', PolynomialFeatures()),
    ('model', LinearRegression()) 
])

# Definimos el grid de parámetros, incluyendo diferentes modelos
param_grid = [
    {
        'polynomialfeatures__degree': degrees,
        'model': [LinearRegression()],
    },
    {
        'polynomialfeatures__degree': degrees,
        'model': [Lasso(max_iter=10000)],
        'model__alpha': lambdas
    },
    {
        'polynomialfeatures__degree': degrees,
        'model': [Ridge(max_iter=10000)],
        'model__alpha': lambdas
    }
]

# Configuramos RepeatedKFold
rkf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=42)

# Configuramos el GridSearchCV con RepeatedKFold
grid_search = GridSearchCV(pipeline, param_grid, cv=rkf, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Mostramos los mejores parámetros encontrados
grid_search.best_params_

/opt/homebrew/Caskroom/miniforge/base/envs/datascience/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.715e+08, tolerance: 4.129e+05
  model = cd_fast.enet_coordinate_descent(
/opt/homebrew/Caskroom/miniforge/base/envs/datascience/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.544e+08, tolerance: 3.542e+05
  model = cd_fast.enet_coordinate_descent(
/opt/homebrew/Caskroom/miniforge/base/envs/datascience/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the nu

{'model': Lasso(max_iter=10000),
 'model__alpha': 100,
 'polynomialfeatures__degree': 2}

In [ ]:
gridfinalresult = PolynomialTrainAndEval(2, X_train, X_test, y_train, y_test, Lasso(alpha=100, max_iter=10000))

In [ ]:
myfinalresult = PolynomialTrainAndEval(2, X_train, X_test, y_train, y_test, Lasso(alpha=10, max_iter=10000))

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

keys = list(gridfinalresult.keys())
fig = make_subplots(rows=1, cols=len(keys), subplot_titles=keys)

for i, key in enumerate(keys):
    fig.add_trace(
        go.Bar(
            x=["Grid Search", "My Model"], 
            y=[gridfinalresult[key], myfinalresult[key]], 
            name=key
        ),
        row=1, col=i + 1
    )

fig.update_layout(
    title_text="Comparación de Resultados entre Grid Search y lo que hicimos",
    xaxis_title="Model",
    yaxis_title="Values",
    showlegend=False 
)

# Mostrar el gráfico
fig.show()

Recordemos que estos son los resultados despues de haber preprocesado los datos eliminando los valroes faltantes y haciendo una selección de atributos utilizando la matriz de correlación y seleccionando las 5 variables más correlacionadas con el precio.